# Programação com sockets: sincronismo e deadlocks

Se não sabe programar com sockets, comece por [Programação com sockets](./02_programacao_com_sockets.ipynb) e [Programação com sockets: protocolos de aplicação](./03_programacao_com_sockets_protocolos_de_aplicacao.ipynb).

Se já sabe, vamos aprofundar o assunto.


### Revisão sobre sockets

No diagrama abaixo temos uma sequência mais acurada dos eventos ocorridos para que dois programas utilizando sockets TCP se comuniquem entre si.

Juntando ambos os diagramas, temos a visão geral do seguinte:
 - a configuração dos sockets do servidor e cliente
 - aguardo do servidor por conexões
 - a abertura da conexão pelo cliente no servidor
   - note que o three-way handshake é feito entre camada de transporte TCP de origem e destino
   - como ambas as aplicações estão em um mesmo servidor, sua pilha de rede é compartilhada
 - o envio de uma requisição do cliente para o servidor
 - a resposta do servidor à requisição do cliente

Note as barras em laranja no cliente e servidor. Elas indicam os momentos em que ambas as aplicações estão BLOQUEADAS.
Ou seja, esperando alguma resposta vinda do sistema operacional.

[![](https://mermaid.ink/img/pako:eNqtVdtO4zAQ_RXLT6xUuu-RFqkbQEKIi0hfVoqEBnugVhs7azvdRYiv4QHxAXxBf4xJ44b0ktIAeYlvOXPmzJn4gQsjkUfc4d8CtcBDBXcWslQzehK0UyWN3T84SJTzmAG7yNGCUEbDJGLOiDH68hTaPgiBud_70WODuwKsBJYby4TR-H_2iq5CBOHVFDzW0NVyPFGoPW6PEw6xX2Her157YTY4vj45Pxr2FrvJRXx6nQyvjgZnxMlSfsopD2F7hU7ADmmvcyBm4UjErlDgDS7hSNyM1CWvPkmlUZCC3bltRB-OLOL-P7hnI9DSjWBMsv85_y6cn4P49Fuwapw2FVvwg4MidqKlEovCXoKFhVNyGpMDs0LT_ux59mTKGRNN8EbQZU_uZv5GuL5FMS39_3tiqJkUMQoQDF35pZZm4cM5mR1bojZebAlzHaCTyRxqucQxiMHAz17Kbr1VNgtazemS11VGJ2iK29h36aAPK94ppTXZ65Sq_1Alu8uNo-6nsVvS-XOWi02-UovKbG2mYEi5uptVY3SwYB360hqBzm1yQnfLfsUOUKu61cgfSbnJDLsgvLdFqEWocKMOW2vQ9hdoMWIdrqF_M_tU8x7PkNRSku7Th3I15X6EGaY8oqEEO055qh_pHBTeJPda8MjbAnu8yCWFDncvj25h4mgVpfLGnlUX9PyefnwDwBbC-A?type=png)](https://mermaid.live/edit#pako:eNqtVdtO4zAQ_RXLT6xUuu-RFqkbQEKIi0hfVoqEBnugVhs7azvdRYiv4QHxAXxBf4xJ44b0ktIAeYlvOXPmzJn4gQsjkUfc4d8CtcBDBXcWslQzehK0UyWN3T84SJTzmAG7yNGCUEbDJGLOiDH68hTaPgiBud_70WODuwKsBJYby4TR-H_2iq5CBOHVFDzW0NVyPFGoPW6PEw6xX2Her157YTY4vj45Pxr2FrvJRXx6nQyvjgZnxMlSfsopD2F7hU7ADmmvcyBm4UjErlDgDS7hSNyM1CWvPkmlUZCC3bltRB-OLOL-P7hnI9DSjWBMsv85_y6cn4P49Fuwapw2FVvwg4MidqKlEovCXoKFhVNyGpMDs0LT_ux59mTKGRNN8EbQZU_uZv5GuL5FMS39_3tiqJkUMQoQDF35pZZm4cM5mR1bojZebAlzHaCTyRxqucQxiMHAz17Kbr1VNgtazemS11VGJ2iK29h36aAPK94ppTXZ65Sq_1Alu8uNo-6nsVvS-XOWi02-UovKbG2mYEi5uptVY3SwYB360hqBzm1yQnfLfsUOUKu61cgfSbnJDLsgvLdFqEWocKMOW2vQ9hdoMWIdrqF_M_tU8x7PkNRSku7Th3I15X6EGaY8oqEEO055qh_pHBTeJPda8MjbAnu8yCWFDncvj25h4mgVpfLGnlUX9PyefnwDwBbC-A)

Agora que vejamos alguns dos problemas que podem ocorrer na programação por sockets.

### Sincronismo e deadlocks

Por padrão, os sockets são blocantes.

Mas o que isto significa? Significa que a thread onde executam permenece bloqueada enquanto uma requisição não é completa.

O diagrama acima mostra o tempo bloqueado com as barras laranjas sobreposta as colunas referentes ao servidor e ao cliente.

Como também pode ser observado no diagrama, estes bloqueios impõem um sincronismo entre as aplicações.

Este sincronismo, por sua vez, pode causar problemas. Um problema típico é o de deadlock.

Um deadlock ocorre quando dois agentes concorrentes tentam obter controle exclusivo sobre um recurso compartilhado.

Para testarmos estes casos, utilizaremos o seguinte lançador do servidor e do cliente:

In [1]:
import socket
import time
from concurrent.futures import ThreadPoolExecutor


def lancador_cliente_servidor(cliente, servidor, cliente2=None):
    thread_pool = ThreadPoolExecutor()
    # Inicia servidor e cliente em threads separadas
    thread_pool.submit(servidor)
    thread_pool.submit(cliente)
    if cliente2:
        thread_pool.submit(cliente2)
    # Dá 10 segundos para cliente e servidor se comunicarem
    time.sleep(15)
    # Mata cliente e servidor
    thread_pool.shutdown(wait=False, cancel_futures=True)


def socket_servidor(porta):
    socket_serv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    socket_serv.bind(("127.0.0.1", porta))
    socket_serv.listen(2)
    return socket_serv


def socket_cliente():
    socket_cli = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    return socket_cli

Quando tratamos de sockets, o deadlock se apresenta de diversas maneiras.
Tipicamente quando são feitas esperas (`socket.recv()` ou `socket.accept()`) em uma ordem incorreta.

[![](https://mermaid.ink/img/pako:eNp9kbFOAzEMhl8l8gTSwQNk6ACsCKRbs1iJAauX-OrkTkJVn4aBB-mLkZKoKkPJ5Py2_8-y9-AlEFjItFsoeXpifFeMLpn6RtKVg-jdZjNyLhTRvMyk6FkSTtZk8Vsqr6j4ODGlQvdKfr25HczDJNWP0eRuYSifOlMQoxXFmY_fxy9pHPSFVyx0Bja5m_5Pv0oW41vqAo0VnmfJBU395T-08xDdsamBrgx3kegNMEAkjcih7nN_qnJQPiiSA1vDgLp14NKh1uFSZPxMHmzRhQZY5lCN-u7BvuGUq0qBi-hzO9DvnQ4_05-ZqQ?type=png)](https://mermaid.live/edit#pako:eNp9kbFOAzEMhl8l8gTSwQNk6ACsCKRbs1iJAauX-OrkTkJVn4aBB-mLkZKoKkPJ5Py2_8-y9-AlEFjItFsoeXpifFeMLpn6RtKVg-jdZjNyLhTRvMyk6FkSTtZk8Vsqr6j4ODGlQvdKfr25HczDJNWP0eRuYSifOlMQoxXFmY_fxy9pHPSFVyx0Bja5m_5Pv0oW41vqAo0VnmfJBU395T-08xDdsamBrgx3kegNMEAkjcih7nN_qnJQPiiSA1vDgLp14NKh1uFSZPxMHmzRhQZY5lCN-u7BvuGUq0qBi-hzO9DvnQ4_05-ZqQ)


In [2]:
def servidor_blocante():
    sock = socket_servidor(8001)
    print("Servidor sendo bloqueado: accept")
    (socketParaCliente, enderecoCliente) = sock.accept()
    print("Servidor sendo desbloqueado: accept")
    print("Servidor sendo bloqueado: recv")
    socketParaCliente.recv(500)
    print("Servidor sendo desbloqueado: recv")
    sock.close()


def cliente_blocante():
    sock = socket_cliente()
    time.sleep(2)  # dar tempo para o servidor ser iniciado
    print("Cliente sendo bloqueado: connect")
    sock.connect(("127.0.0.1", 8001))
    print("Cliente sendo desbloqueado: connect")
    print("Cliente sendo bloqueado: recv")
    sock.recv(500)
    print("Cliente sendo desbloqueado: recv")
    sock.close()


lancador_cliente_servidor(cliente_blocante, servidor_blocante)

Servidor sendo bloqueado: accept
Cliente sendo bloqueado: connect
Cliente sendo desbloqueado: connect
Cliente sendo bloqueado: recv
Servidor sendo desbloqueado: accept
Servidor sendo bloqueado: recv


Como se pode observar, ambos ficaram bloqueados aguardando a resposta um do outro.

O mesmo pode acontecer com accept e recv.
[![](https://mermaid.ink/img/pako:eNp9UTFOxEAM_MrKFUjhHpDiCqBFFNemsbxzYF2yDrubCHS611DwAl6Qj7EhAR1IsJV3Zjxj2UcS86CaEp4GBMGt8kPkrgmuvB3iqN7i1Xa705TRsbvvEVnUAre1SyYH5C_VhkXQ54vLyl23VuyUXVo5hzQ3Bm-uLz-xgOfpHWnJYck6csZ34ALftIqQ8X_6KtpEyPgj2pws1Fk2z8l7jR1Pb9OruQIhjGq_xlg9F9Tjj_HOiLWBKupQzNWXjR5nVUP5ER0aqkvpOR4aasKp6HjItnsJQnWOAyoael-M1u1Tvec2FRRes8W75USflzp9ACCOmtw?type=png)](https://mermaid.live/edit#pako:eNp9UTFOxEAM_MrKFUjhHpDiCqBFFNemsbxzYF2yDrubCHS611DwAl6Qj7EhAR1IsJV3Zjxj2UcS86CaEp4GBMGt8kPkrgmuvB3iqN7i1Xa705TRsbvvEVnUAre1SyYH5C_VhkXQ54vLyl23VuyUXVo5hzQ3Bm-uLz-xgOfpHWnJYck6csZ34ALftIqQ8X_6KtpEyPgj2pws1Fk2z8l7jR1Pb9OruQIhjGq_xlg9F9Tjj_HOiLWBKupQzNWXjR5nVUP5ER0aqkvpOR4aasKp6HjItnsJQnWOAyoael-M1u1Tvec2FRRes8W75USflzp9ACCOmtw)


In [3]:
def servidor_blocante2():
    sock = socket_servidor(8002)
    print("Servidor sendo bloqueado: accept")
    (socketParaCliente, enderecoCliente) = sock.accept()
    print("Servidor sendo desbloqueado: accept")
    socketParaCliente.close()
    print("Servidor sendo bloqueado: accept")
    (socketParaCliente, enderecoCliente) = sock.accept()
    print("Servidor sendo desbloqueado: accept")
    sock.close()


def cliente_blocante2():
    sock = socket_cliente()
    time.sleep(2)  # dar tempo para o servidor ser iniciado
    print("Cliente sendo bloqueado: connect")
    sock.connect(("127.0.0.1", 8002))
    print("Cliente sendo desbloqueado: connect")
    print("Cliente sendo bloqueado: recv")
    sock.recv(500)
    print("Cliente sendo desbloqueado: recv")
    sock.close()


lancador_cliente_servidor(cliente_blocante2, servidor_blocante2)

Servidor sendo bloqueado: accept
Cliente sendo bloqueado: connect
Cliente sendo desbloqueado: connect
Cliente sendo bloqueado: recv
Servidor sendo desbloqueado: accept
Servidor sendo bloqueado: accept
Cliente sendo desbloqueado: recv


Este é um problema relativalmente comum para programadores iniciantes.
Ocorre quando tentam expandir o número de clientes conectados simultâneamente a um servidor.

Pior, o deadlock pode se propagar para outros clientes.

[![](https://mermaid.ink/img/pako:eNqNkTGLAkEMhf_KkkrBs9hyCpuzPU7Ydpow87wb3JnRbHZBxP_u6C6CnOilCi8vHyTvRC57kKEOhx7JYR34RzjaVJVqIEPwWT5WqyZ0isjV9x7CLuTEram67HbQDQt_tgFJUS8FbpjNx3V2GgZW3DmjPHlfQyfTc940fMDV_-LVL4H1KHu8Hzye9HeDFhQhkYMvzz1dXZb0FxGWTGk9y86STefi415zc0yOjEqPBfV7X0BTEGS23HZFhQ-a5WtM6xba-QKor5mH?type=png)](https://mermaid.live/edit#pako:eNqNkTGLAkEMhf_KkkrBs9hyCpuzPU7Ydpow87wb3JnRbHZBxP_u6C6CnOilCi8vHyTvRC57kKEOhx7JYR34RzjaVJVqIEPwWT5WqyZ0isjV9x7CLuTEram67HbQDQt_tgFJUS8FbpjNx3V2GgZW3DmjPHlfQyfTc940fMDV_-LVL4H1KHu8Hzye9HeDFhQhkYMvzz1dXZb0FxGWTGk9y86STefi415zc0yOjEqPBfV7X0BTEGS23HZFhQ-a5WtM6xba-QKor5mH)


In [4]:
def servidor_blocante3():
    sock = socket_servidor(8003)
    for i in range(2):
        print("Servidor sendo bloqueado: accept")
        (socketParaCliente, enderecoCliente) = sock.accept()
        print("Servidor sendo desbloqueado: accept")
        print("Servidor sendo bloqueado: recv")
        socketParaCliente.recv(500)
        print("Servidor sendo desbloqueado: recv")
    sock.close()


def cliente_blocante3(id):
    sock = socket_cliente()
    time.sleep(2)  # dar tempo para o servidor ser iniciado
    print(f"Cliente {id} sendo bloqueado: connect")
    sock.connect(("127.0.0.1", 8003))
    print(f"Cliente {id} sendo desbloqueado: connect")
    print(f"Cliente {id} sendo bloqueado: recv")
    sock.recv(500)
    print(f"Cliente {id} sendo desbloqueado: recv")
    sock.close()


from functools import partial

lancador_cliente_servidor(partial(cliente_blocante3, ""), servidor_blocante3, partial(cliente_blocante3, "2"))

Servidor sendo bloqueado: accept
Cliente 2 sendo bloqueado: connect
Servidor sendo desbloqueado: accept
Servidor sendo bloqueado: recv
Cliente  sendo bloqueado: connect
Cliente 2 sendo desbloqueado: connect
Cliente 2 sendo bloqueado: recv
Cliente  sendo desbloqueado: connect
Cliente  sendo bloqueado: recv


Como lidar com este tipo de problema?

A solução típica é separar estas chamadas blocantes em threads distintas.

In [5]:
from threading import Thread


def cliente_blocante4(id, porta=8004):
    sock = socket_cliente()
    time.sleep(4)  # dar tempo para o servidor ser iniciado
    print(f"Cliente {id} sendo bloqueado: connect")
    sock.connect(("127.0.0.1", porta))
    print(f"Cliente {id} sendo desbloqueado: connect")
    print(f"Cliente {id} sendo bloqueado: recv")
    sock.recv(500)
    print(f"Cliente {id} sendo desbloqueado: recv")
    sock.close()


def cliente_blocante4_2(id, porta=8004):
    sock = socket_cliente()
    time.sleep(4)  # dar tempo para o servidor ser iniciado
    print(f"Cliente {id} sendo bloqueado: connect")
    sock.connect(("127.0.0.1", porta))
    print(f"Cliente {id} sendo desbloqueado: connect")
    print(f"Cliente {id} sendo bloqueado: send")
    sock.send(bytes("pindamonhangaba", "utf-8"))
    print(f"Cliente {id} sendo desbloqueado: send")
    print(f"Cliente {id} sendo bloqueado: recv")
    sock.recv(500)
    print(f"Cliente {id} sendo desbloqueado: recv")
    sock.close()


class servidor_blocante4:
    def __init__(self, endereco_servidor="0.0.0.0", porta_servidor=8004, max_conexoes=1):
        # Procedimento de criação do socket e configuração
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.bind((endereco_servidor, porta_servidor))
        self.socket.listen(max_conexoes)

        # Inicia uma thread dedicada para escuta de novas conexões
        self.threadEscuta = Thread(target=self.implementacaoThreadEscuta)
        self.threadEscuta.run()
        self.threadClientes = []

    def implementacaoThreadCliente(self, enderecoDoCliente, socketParaCliente):
        max_messages = 3
        while max_messages > 0:
            print("Servidor sendo bloqueado: recv")
            mensagem = socketParaCliente.recv(512)
            print("Servidor sendo desbloqueado: recv")
            print("Servidor sendo bloqueado: send")
            socketParaCliente.send(mensagem)
            print("Servidor sendo bloqueado: send")
            max_messages -= 1

    def implementacaoThreadEscuta(self):
        while True:
            # Thread fica bloqueada enquanto aguarda por conexões,
            # enquanto servidor continua rodando normalmente
            print("Servidor sendo bloqueado: accept")
            (socketParaCliente, enderecoDoCliente) = self.socket.accept()
            print("Servidor sendo desbloqueado: accept")
            novaThread = Thread(target=self.implementacaoThreadCliente,
                                args=(enderecoDoCliente, socketParaCliente),
                                daemon=True)  # thread sem necessidade de join, será morta ao final do processo
            novaThread.run()  # inicia thread de atendimento ao novo cliente conectado
            self.threadClientes.append(novaThread)


lancador_cliente_servidor(partial(cliente_blocante4, "", 8004),
                          partial(servidor_blocante4, porta_servidor=8004),
                          partial(cliente_blocante4_2, "2", 8004))

Servidor sendo bloqueado: accept
Cliente 2 sendo bloqueado: connect
Cliente  sendo bloqueado: connect
Cliente 2 sendo desbloqueado: connect
Cliente 2 sendo bloqueado: send
Servidor sendo desbloqueado: accept
Servidor sendo bloqueado: recv
Servidor sendo desbloqueado: recv
Servidor sendo bloqueado: send
Cliente 2 sendo desbloqueado: send
Cliente 2 sendo bloqueado: recv
Cliente 2 sendo desbloqueado: recv
Servidor sendo bloqueado: send
Servidor sendo bloqueado: recv
Cliente  sendo desbloqueado: connect
Cliente  sendo bloqueado: recv
Servidor sendo desbloqueado: recv
Servidor sendo bloqueado: send
Servidor sendo bloqueado: send
Servidor sendo bloqueado: recv
Servidor sendo desbloqueado: recv
Servidor sendo bloqueado: send
Servidor sendo bloqueado: send


Existe outra alternativa invés de ficar brigando com a ordenação e arriscar entrar em deadlock.

Podem ser configurados timeouts, onde uma condição de erro é disparada e o programador reassume controle do programa.

In [6]:
def servidor_blocante6():
    sock = socket_servidor(8006)
    sock.settimeout(3)  # 3 segundos
    socketParaCliente = None
    enderecoCliente = None
    retries = 3
    while retries > 0:
        print("Servidor sendo bloqueado: accept")
        try:
            (socketParaCliente, enderecoCliente) = sock.accept()
            break
        except TimeoutError:
            print("Servidor sendo desbloqueado devido a timeout: accept")
            retries -= 1
    print("Servidor sendo desbloqueado: accept")
    socketParaCliente.settimeout(3)  # 3 segundos
    retries = 3
    while retries > 0:
        print("Servidor sendo bloqueado: recv")
        try:
            socketParaCliente.recv(500)
            break
        except TimeoutError:
            print("Servidor sendo desbloqueado devido a timeout: recv")
            retries -= 1
    print("Servidor sendo desbloqueado: recv")
    sock.close()


def cliente_blocante6():
    sock = socket_cliente()
    sock.settimeout(3)  # 3 segundos
    time.sleep(2)  # dar tempo para o servidor ser iniciado
    print("Cliente sendo bloqueado: connect")
    sock.connect(("127.0.0.1", 8006))
    print("Cliente sendo desbloqueado: connect")
    retries = 3
    while retries >= 3:
        print("Cliente sendo bloqueado: recv")
        try:
            sock.recv(500)
            break
        except TimeoutError:
            print("Cliente sendo desbloqueado devido a timeout: recv")
            time.sleep(1)
            retries -= 1
    print("Cliente sendo desbloqueado: recv")
    sock.close()


lancador_cliente_servidor(cliente_blocante6, servidor_blocante6)

Servidor sendo bloqueado: accept
Cliente sendo bloqueado: connect
Cliente sendo desbloqueado: connect
Cliente sendo bloqueado: recv
Servidor sendo desbloqueado: accept
Servidor sendo bloqueado: recv
Cliente sendo desbloqueado devido a timeout: recvServidor sendo desbloqueado devido a timeout: recv
Servidor sendo bloqueado: recv

Cliente sendo desbloqueado: recv
Servidor sendo desbloqueado: recv


Outra opção é o uso de sockets não blocantes.

Ao invés de o processo que quer receber uma mensagem ficar bloqueado e ser acordado pelo sistema operacional para processar a mensagem (interrupção),
o processo fica acordado e pergunta para o sistema operacional se já chegou alguma mensagem (polling).

[![](https://mermaid.ink/img/pako:eNqtkjFOAzEURK9iuQJpyQFcpIEWEbHtNj_27K6VtR2-7ZUgymEQR8nFcFgrFKA0YLmwvp9m_khzkDoYSCUjXjK8xoOlgcl1XpTTgmdrAt-t162NCY7E0x5M2gZPkxIx6B3ShpjuJwufsLIMPd_cNmIDHrJPJCLEeHoXDj7SACcKgK01tDiQTnamhItVNf7pdt6hMko8IwX2JHJhLsrlbnPfg1dFTfjTRxBjyDO4-WZmerO0WkwMfnFfvmqc67n_nLkKXItckf9PXIVlIx3YkTWlA4cz1ck0wqGTqjwN8a6TnT8WjnIK7avXUiXOaGTemyJU-yJVT1MsUxhbFn1cSvXVreMn_I3Y_g?type=png)](https://mermaid.live/edit#pako:eNqtkjFOAzEURK9iuQJpyQFcpIEWEbHtNj_27K6VtR2-7ZUgymEQR8nFcFgrFKA0YLmwvp9m_khzkDoYSCUjXjK8xoOlgcl1XpTTgmdrAt-t162NCY7E0x5M2gZPkxIx6B3ShpjuJwufsLIMPd_cNmIDHrJPJCLEeHoXDj7SACcKgK01tDiQTnamhItVNf7pdt6hMko8IwX2JHJhLsrlbnPfg1dFTfjTRxBjyDO4-WZmerO0WkwMfnFfvmqc67n_nLkKXItckf9PXIVlIx3YkTWlA4cz1ck0wqGTqjwN8a6TnT8WjnIK7avXUiXOaGTemyJU-yJVT1MsUxhbFn1cSvXVreMn_I3Y_g)


In [7]:
def servidorNaoBlocante():
    sock = socket_servidor(8007)
    sock.setblocking(False)
    socketParaCliente = None
    enderecoCliente = None
    while enderecoCliente is None:
        time.sleep(1)
        print("Servidor aceitando conexão")
        try:
            res = sock.accept()
            (socketParaCliente, enderecoCliente) = res
        except:
            pass
    print("Servidor recebeu requisição de conexão de:", enderecoCliente)
    msg = ""
    retries = 3
    while retries > 0 and len(msg) == 0:
        print("Servidor tentando receber mensagem do cliente:", enderecoCliente)
        msg = socketParaCliente.recv(500)
        retries -= 1
    if retries == 0:
        print("Servidor não recebeu nada")
    else:
        print("Servidor recebeu:", msg)
    sock.close()


def clienteNaoBlocante():
    sock = socket_cliente()
    sock.setblocking(False)
    time.sleep(2)  # dar tempo para o servidor ser iniciado
    print("Cliente requisitando conexão")
    sock.connect(("127.0.0.1", 8007))
    print("Cliente conectado")
    msg = sock.recv(500)
    print("Cliente recebeu:", msg)
    sock.close()


lancador_cliente_servidor(clienteNaoBlocante, servidorNaoBlocante)

Servidor aceitando conexão
Cliente requisitando conexão
Servidor aceitando conexão
Servidor recebeu requisição de conexão de: ('127.0.0.1', 61472)
Servidor tentando receber mensagem do cliente: ('127.0.0.1', 61472)


Este tipo de abordagem permite menor tempo de resposta em aplicações de alto desempenho/baixa latência, porém ao custo de desperdiçar tempo de CPU.

[![](https://mermaid.ink/img/pako:eNrtlE1KAzEcxa8SslIYe4AsClJdiVic7Wz-Td5MQydJzceAlh7Gs3gxMzaOYKUIbkQMWYTw8t4vIbwdl06BCx7wkGAlrjR1nkxjWR41_KCV8xfzea1DhCF2t4UnqZ2lXrDg5AZxSZ4WvYaNmGkPOZydV2wJ3yUbiQWw9cszM7CBOhiWBVhpRYcEklEPFDFFleDjtJGhaAS7R3TeEktZMzkP9KRpdjBQ-ML5h5f6J_42ceE4DfwOG2DV-GWu7aA_oj6xF_Ep9CIR7HJxc0Q4nf91T5rnKrUt_F_6CLziBt6QVrlbdqOs4XENg4aLvFTkNw1v7D7rKEVXP1rJRfQJFU9blZ1KD3HRUh_yLpTOFLeHsnrrrP0rbNCrDg?type=png)](https://mermaid.live/edit#pako:eNrtlE1KAzEcxa8SslIYe4AsClJdiVic7Wz-Td5MQydJzceAlh7Gs3gxMzaOYKUIbkQMWYTw8t4vIbwdl06BCx7wkGAlrjR1nkxjWR41_KCV8xfzea1DhCF2t4UnqZ2lXrDg5AZxSZ4WvYaNmGkPOZydV2wJ3yUbiQWw9cszM7CBOhiWBVhpRYcEklEPFDFFleDjtJGhaAS7R3TeEktZMzkP9KRpdjBQ-ML5h5f6J_42ceE4DfwOG2DV-GWu7aA_oj6xF_Ep9CIR7HJxc0Q4nf91T5rnKrUt_F_6CLziBt6QVrlbdqOs4XENg4aLvFTkNw1v7D7rKEVXP1rJRfQJFU9blZ1KD3HRUh_yLpTOFLeHsnrrrP0rbNCrDg)